# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
keras = tf.keras
from sklearn.preprocessing import RobustScaler
import plotly.graph_objects as go
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

# Date Pre-Processing

In [2]:
confirmed_df = pd.read_csv('dataset_final.csv')
confirmed_df.head()

confirmed_df = confirmed_df.groupby("Province/State").sum().reset_index()
print("DONE 1")
Country_confirmed =  confirmed_df["Province/State"].values
print("DONE 2")
confirmed_df = confirmed_df.drop(['Province/State'], axis=1)
print("DONE 3")
confirmed_df = confirmed_df[confirmed_df.columns].T
print("DONE 4")
confirmed_df.columns = Country_confirmed
print("DONE 5")
Date_index = confirmed_df.index
print("DONE 6")
confirmed_df = confirmed_df.diff().fillna(0)
print("DONE 7")
confirmed_df = confirmed_df.abs()
print("DONE 8")
confirmed_df.head()


DONE 1
DONE 2
DONE 3
DONE 4
DONE 5
DONE 6
DONE 7
DONE 8


,AB,AG,AR,B,BC,BH,BN,BR,BT,BV,...,SB,SJ,SM,SV,TL,TM,TR,VL,VN,VS
04/03/2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
04/04/2020,2.0,2.0,18.0,6.0,3.0,2.0,1.0,2.0,4.0,3.0,...,0.0,2.0,1.0,101.0,0.0,10.0,3.0,1.0,0.0,0.0
04/05/2020,18.0,11.0,22.0,2.0,11.0,0.0,14.0,0.0,20.0,4.0,...,22.0,3.0,6.0,248.0,0.0,40.0,0.0,1.0,9.0,2.0
04/06/2020,1.0,2.0,7.0,14.0,1.0,6.0,16.0,0.0,16.0,7.0,...,6.0,2.0,4.0,13.0,0.0,5.0,9.0,0.0,0.0,1.0
04/07/2020,16.0,4.0,4.0,32.0,8.0,3.0,12.0,1.0,2.0,18.0,...,14.0,1.0,2.0,94.0,0.0,10.0,1.0,1.0,0.0,1.0


# Scalling the data

In [3]:
sc = RobustScaler()
confirmed_df_scaled = sc.fit_transform(confirmed_df)
confirmed_df_scaled = pd.DataFrame(confirmed_df_scaled)

# Enter the number of days you want to predict

In [4]:
DAYS_TO_PREDICT = int(input('Enter Number Of Days You Want to Predict : '))
predict_confirm_index = pd.date_range(start=Date_index[-1],periods=DAYS_TO_PREDICT + 1,closed='right')
predict_confirm_index = predict_confirm_index.strftime('%Y/%m/%d')

Enter Number Of Days You Want to Predict : 7


# Creating class for building model

In [5]:
class Corona_Confirmed_model():
    def create_dataset(self, dataset, time_steps=1, column = 0):
        dataX, dataY = [], []
        for i in range(len(dataset)-time_steps-1):
          a = dataset[i:(i+time_steps), column]
          dataX.append(a)
          dataY.append(dataset[i + time_steps, column])
        return np.array(dataX), np.array(dataY)

    def Model(self,dframe,column):
        df = dframe.values
        df = df.astype('float32')
        
        time_steps = 1
        trainX, trainY = self.create_dataset(df, time_steps, column)
        trainX = np.reshape(trainX, (trainX.shape[0],1,trainX.shape[1]))
        
        
        keras.backend.clear_session()
        tf.random.set_seed(42)
        np.random.seed(42)
        confirm_model = keras.models.Sequential([
            keras.layers.InputLayer(input_shape=[None, 1]),
            keras.layers.Conv1D(filters=16, kernel_size=2,
                      strides=1, padding="causal",
                      activation="relu",
                      ),
            keras.layers.LSTM(30,kernel_initializer = 'he_normal',
                              return_sequences = True),
            keras.layers.LSTM(30,kernel_initializer = 'he_normal',
                              return_sequences = True),
            keras.layers.Dense(15,kernel_initializer = 'he_normal',
                                             activation = 'relu'),
            keras.layers.Dense(15,kernel_initializer = 'he_normal',
                                             activation = 'relu'),
            keras.layers.Dense(1,activation = 'linear',kernel_initializer = 'he_normal'),
            keras.layers.Lambda(lambda x: x * 200)
             ])
        
        optimizer = keras.optimizers.Adadelta(lr=0.001)
        
        confirm_model.compile(loss=['mae'],optimizer=optimizer)   
                
        confirm_model.fit(trainX, trainY, epochs=100,
                          batch_size = 10,verbose = 0)
        

        confirm_test_seq  = trainX[-1:]
        confirm_test_seq = confirm_test_seq
        confirm_test_seq = confirm_test_seq.reshape(len(confirm_test_seq), time_steps, 1)
        confirm_preds = []
        
        for _ in range (DAYS_TO_PREDICT):
            confirm_pred = confirm_model.predict(confirm_test_seq)
            confirm_preds.append(confirm_pred)
            confirm_new_seq = confirm_pred
            confirm_test_seq = confirm_new_seq
            confirm_test_seq = confirm_test_seq.reshape(len(confirm_test_seq ), time_steps, 1)

        

        return confirm_preds

# Calling the class and predicting the cases for every individual country

In [6]:
NN = Corona_Confirmed_model()
confirm_predictions = []
for i in range(len(Country_confirmed)):
    result = NN.Model(confirmed_df_scaled,i) 
    confirm_predictions.append(result)
    print(i)
    


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41


# Processing the predicted data

In [7]:
confirm_predictions = (np.array(confirm_predictions).transpose())
confirm_predictions = confirm_predictions.reshape(((DAYS_TO_PREDICT),len(Country_confirmed)))
confirm_predictions = sc.inverse_transform(confirm_predictions)
confirm_predictions = pd.DataFrame(confirm_predictions)
confirm_predictions = confirm_predictions.abs()
confirm_predictions['Dates'] = predict_confirm_index
confirm_predictions.set_index('Dates', inplace = True)
confirm_predictions.columns = Country_confirmed
confirm_predictions.head()

Dates
2020/10/29    523.862854
2020/10/30    525.524780
2020/10/31    526.226807
2020/11/01    526.522461
2020/11/02    526.645935
2020/11/03    526.697144
2020/11/04    526.718872
Name: B, dtype: float32

# Visualizing The Confirmed Cases For A Country As Per The User's Choice

### List of Infected Countries

In [8]:
Infected_Countries = pd.DataFrame(Country_confirmed)
Infected_Countries.columns = ['Countries']
pd.set_option("max_rows", None)
Infected_Countries

,Countries
0,AB
1,AG
2,AR
3,B
4,BC
5,BH
6,BN
7,BR
8,BT
9,BV


### Select The Country

In [25]:
Country =input('Enter the name of the country from above list you would like to see the daily cases till now : ')

Enter the name of the country from above list you would like to see the daily cases till now : AB


## Till Date Confirm Cases on daily basis of coronavirus

In [26]:
fig = go.Figure()
fig.update_layout(template='plotly_dark',title={'text': 'Confirmed Daily Cases','y':0.9,'x':0.5,'xanchor': 'center',
                                                'yanchor': 'top'},
                    xaxis_title = "Dates",
                    yaxis_title="Number Of Cases",
                     )
fig.add_trace(go.Scatter(x=confirmed_df.index, 
                         y=confirmed_df[Country],
                         mode='lines+markers',
                         line=dict(color='blue', width=2)))

## Predictions of confirmed cases of coronavirus

In [27]:
fig = go.Figure()
fig.update_layout(template='plotly_dark',title={'text': 'Confirmed Cases Predictions','y':0.9,'x':0.5,'xanchor': 'center',
                                                'yanchor': 'top'},
    xaxis_title = "Dates",
    yaxis_title="Number Of Cases")
fig.update_xaxes(tickangle=90, showticklabels = True, type = 'category')
fig.add_trace(go.Scatter(x=confirm_predictions.index, 
                         y=confirm_predictions[Country],
                         mode='lines+markers',
                         line=dict(color='yellow', width=2)))

In [33]:
county_list = Infected_Countries.values.tolist()
print(lista_judete)

f = open("predictions.txt", "a+")

for county in county_list:
    f.write(str(confirm_predictions[county]))
    #print(str(confirm_predictions[county]))
    
f.close()


[['AB'], ['AG'], ['AR'], ['B'], ['BC'], ['BH'], ['BN'], ['BR'], ['BT'], ['BV'], ['BZ'], ['CJ'], ['CL'], ['CS'], ['CT'], ['CV'], ['DB'], ['DJ'], ['GJ'], ['GL'], ['GR'], ['HD'], ['HR'], ['IF'], ['IL'], ['IS'], ['MH'], ['MM'], ['MS'], ['NT'], ['OT'], ['PH'], ['SB'], ['SJ'], ['SM'], ['SV'], ['TL'], ['TM'], ['TR'], ['VL'], ['VN'], ['VS']]
